<a href="https://colab.research.google.com/github/MedAljane/Stage_Project_files/blob/main/coronas_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deep-translator

In [2]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator
import json
import pandas as pd
from datetime import datetime

In [3]:
today = datetime.today().strftime('%Y-%m-%d')

In [4]:
#Getting available countries list for help
countrieslinks = []
links = []
aTags = BeautifulSoup(requests.get("https://www.worldometers.info/coronavirus/#countries").content, 'html.parser').find('table', {'id': 'main_table_countries_today'}).find('tbody').findAll('a', {'class': 'mt_a'})
for a in aTags:
  links.append(a['href'][a['href'].index('/')+1:-1])
  countrieslinks.append(f"Insert ==> {a['href'][a['href'].index('/')+1:-1]} <== for: {a.text}")

In [ ]:
#Getting the equivalent argument for each country to use in the GET request
link_DataCountry = {}
for counter, link in enumerate(links):
  try:
    tgt = BeautifulSoup(requests.get("https://www.worldometers.info/coronavirus/country/"+link+"/").content, 'html.parser').find('a', {'class': 'load-more__btn'})
    link_DataCountry[link] = tgt['data-country']
    print(f"({counter}/{len(links)}) ** {link} Done! moving to next...")
  except:
    print(f"({counter}/{len(links)}) ** {link} Not found! moving to next...")

In [ ]:
#Inputing the desired country and getting its latest cases
paysTarget = input('Enter country name: ').lower()
try:
    soup = BeautifulSoup(requests.get(f"https://www.worldometers.info/coronavirus/news-block/news_main_updates.php?fd=lm_{today}&country={link_DataCountry[paysTarget]}&days_count=14").content, 'html.parser').findAll('button', {'class': 'btn btn-light date-btn'})
    dates_En = []
    cases_En = []
    for date in soup:
        c = date.find_next_sibling('div', {'class': 'newsdate_div'}).findChild('li', {'class': 'news_li'})
        cases_En.append(c.text[:c.text.index('in')].strip())
        dates_En.append(date.text.strip())    

    new_cases_En = []
    new_deaths_En = []

    for e in cases_En:
        if "new cases" in e and "new deaths" in e:
            new_cases_En.append(e.split('and')[0].strip())
            new_deaths_En.append(e.split('and')[1].strip())
        elif "new cases" in e:
            new_cases_En.append(e)
            new_deaths_En.append("No new deaths")
        elif "new deaths" in e:
            new_cases_En.append("No new cases")
            new_deaths_En.append(e)
        else:
            new_cases_En.append("No new cases")
            new_deaths_En.append("No new deaths")

    new_cases_Fr = [GoogleTranslator(source='en', target='fr').translate(e) for e in new_cases_En]
    new_deaths_Fr = [GoogleTranslator(source='en', target='fr').translate(e) for e in new_deaths_En]
    dates_Fr = [GoogleTranslator(source='en', target='fr').translate(e) for e in dates_En]

    coronaDict_En = {
        "date": dates_En,
        "new_cases": new_cases_En,
        "new_deaths": new_deaths_En
    }
    df_En = pd.DataFrame(coronaDict_En)
    print("Anglais:\n", df_En)

    coronaDict_Fr = {
        "date": dates_Fr,
        "nouveaux_cas": new_cases_Fr,
        "nouveaux_décès": new_deaths_Fr
    }
    df_Fr = pd.DataFrame(coronaDict_Fr)
    print("Francais:\n", df_Fr)

except Exception as e:
    print(e, '\nCountry not in list!\nTo know what to put check ==> countrieslinks <== variable!')

In [8]:
#Exporter le resultat obtenu en tant que JSON

df_Fr.to_json('coronaCases_Fr.json', orient='records', lines=True, force_ascii=False)
df_En.to_json('coronaCases_En.json', orient='records', lines=True, force_ascii=False)


#If you want to include the name of the country in the filename uncomment the following 2 lines and comment the previous 2

#df_Fr.to_json(f'{paysTarget}_coronaCases_Fr.json', orient='records', lines=True, force_ascii=False)
#df_En.to_json(f'{paysTarget}_coronaCases_En.json', orient='records', lines=True, force_ascii=False)